In [1]:
import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os

In [2]:
import pandas as pd 
print(np.__version__)

1.24.4


## LOAD DATA

In [2]:
def load_Dataset(baseFile, fold_Number): 
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    base_File = f"{baseFile}/u{fold_Number}.base"
    test_File = f"{baseFile}/u{fold_Number}.test"
    base_Data = pd.read_csv(base_File, sep="\t", header=None, names=rnmColData)
    test_Data = pd.read_csv(test_File, sep="\t", header=None, names=rnmColData)
    base_Data = base_Data.drop(columns=["timestamp"])
    test_Data = test_Data.drop(columns=["timestamp"])
    return base_Data, test_Data

In [3]:
call_base = "ml-100k"
basedata, testdata = load_Dataset(call_base, 1)
basedata

,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


## CONVERT RATING MATRIKS

In [4]:
def ConvertRatingMatriks(ratingData):
    # membuat container untuk rating matriks dengan ukuran user x item
    matriks_rating = pd.DataFrame(np.zeros((943, 1682)), columns = list(range(1, 1683)), index = list(range(1, 944)))
    # merubah data frame ke dalam bentuk matriks rating pivot
    convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
    # mengisi matriks rating NaN dengan 0
    convertMatriksRating = convertMatriksRating.fillna(0)
    # updating matriks rating dengan matriks rating yang sudah di pivot
    matriks_rating.update(convertMatriksRating)
    # mengembalikan matriks rating
    return matriks_rating

## BASE MODEL

In [6]:
rating_Matriks = ConvertRatingMatriks(basedata)
rating_Matriks

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### MEAN 

In [7]:
def meanRating(RatingMatriks, jenis="user-based"):
    if jenis == "user-based":
        # menghitung berapa banyak rating yang ada pada setiap user(baris)
        axis = 1
        # index baris user dari rating matriks
        index = RatingMatriks.index
    elif jenis == "item-based":
        # menghitung berapa banyak rating yang ada pada setiap item(kolom)
        axis = 0
        # index kolom item dari rating matriks
        index = RatingMatriks.columns
    # menghitung mean rating dari setiap user/item
    pembilang = RatingMatriks.sum(axis=axis)
    # menghitung banyaknya rating yang ada pada setiap user/item yang bukan 0
    penyebut = np.count_nonzero(RatingMatriks, axis=axis)
    # cek jika penyebut == 0, maka mean rating = 0, jika tidak hitung mean rating
    calculateMeanRating = np.where(penyebut == 0, 0, pembilang / penyebut)
    # mengembalikan mean rating dalam bentuk data frame
    meanRatingNew = pd.DataFrame(calculateMeanRating, index=index, columns=["meanRating"])
    return meanRatingNew

#### USER

In [8]:
cal_MeanRatingUser = meanRating(rating_Matriks, jenis="user-based")
cal_MeanRatingUser

,meanRating
1,3.681481
2,3.800000
3,3.000000
4,4.357143
5,2.956044
...,...
939,4.265306
940,3.457944
941,4.045455
942,4.265823


#### ITEM

In [9]:
cal_MeanRatingItem = meanRating(rating_Matriks, jenis="item-based")
cal_MeanRatingItem

,meanRating
1,3.892950
2,3.180952
3,3.000000
4,3.526316
5,3.304348
...,...
1678,1.000000
1679,3.000000
1680,2.000000
1681,3.000000


### MEAN-CENTERED

In [10]:
def meanCenteredRating(RatingMatriks, meanRating, jenis="user-based"):
    # matriks rating jadi numpy array
    npRatingMatriks = np.array(RatingMatriks)
    if jenis == "user-based":
        # Reshape meanRating menjadi (jumlah_user, 1), kolom
        npMeanRating = np.array(meanRating).reshape(-1, 1)  # (943, 1)
    elif jenis == "item-based":
        # Reshape meanRating menjadi (1, jumlah_item), baris
        npMeanRating = np.array(meanRating).reshape(1, -1)  # (1, 1650)
    # Menghitung mean centered, hanya mengurangi rating yang bukan 0, jika rating 0 maka hasilnya 0
    meanCentered = np.where(npRatingMatriks != 0, npRatingMatriks - npMeanRating, 0)
    # Mengembalikan dalam bentuk DataFrame
    dfMeanCentered = pd.DataFrame(meanCentered, index=RatingMatriks.index, columns=RatingMatriks.columns)
    return dfMeanCentered

#### USER

In [11]:
cal_MeanCenteredUser = meanCenteredRating(rating_Matriks, cal_MeanRatingUser, jenis="user-based")
cal_MeanCenteredUser

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.318519,-0.681481,0.318519,-0.681481,-0.681481,0.0,0.318519,-2.681481,1.318519,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.734694,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.000000,0.000000,0.000000,-1.457944,0.000000,0.0,0.542056,1.542056,-0.457944,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.954545,0.000000,0.000000,0.000000,0.000000,0.0,-0.045455,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### ITEM

In [12]:
cal_MeanCenteredItem = meanCenteredRating(rating_Matriks, cal_MeanRatingItem, jenis="item-based")
cal_MeanCenteredItem

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.10705,-0.180952,1.0,-0.526316,-0.304348,0.0,0.201954,-2.99422,1.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.10705,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,-1.876712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,1.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.00000,0.000000,0.0,-1.526316,0.000000,0.0,0.201954,1.00578,-0.833333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,1.10705,0.000000,0.0,0.000000,0.000000,0.0,0.201954,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### SIMILARITY

In [ ]:
def SimilarityRJ(RatingMatriks, index1, index2, jenis="user-based"):
    # jika jenis == "user-based":
    if jenis == "user-based":
        Rated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] != 0])
        # print("itemRated1: ", itemRated1)
        Rated2 = set(RatingMatriks.columns[RatingMatriks.iloc[index2, :] != 0])
        # print("Rated2: ", Rated2)
    elif jenis == "item-based":
        # mencari index user yang telah memberi rating pada item1 dan item2
        Rated1 = set(RatingMatriks.index[RatingMatriks.iloc[:, index1] != 0])
        Rated2 = set(RatingMatriks.index[RatingMatriks.iloc[:, index2] != 0])
    # menghitung banyaknya item yang telah diberi rating oleh user1 dan user2
    intersection = len(Rated1.intersection(Rated2))
    #menghitung jumlah total item yang telah diberi rating oleh user dan jumlah total item yang telah diberi rating oleh user
    CountRated1 = len(Rated1)
    CountRated2 = len(Rated2)
    # menghitung similarity
    if intersection != 0:
        rumusSimRJItem = 1/(1+(1/intersection)+(CountRated1/1+CountRated1)+(CountRated2/1+CountRated2))
    elif intersection == 0:
        rumusSimRJItem = 0
    return rumusSimRJItem

In [14]:
cal_simRJUser = SimilarityRJ(rating_Matriks, 0, 0, jenis="user-based")
cal_simRJUser

0.0018484035270277671

In [15]:
cal_simRJItem = SimilarityRJ(rating_Matriks, 1, 2, jenis="item-based")
cal_simRJItem

0.0072815533980582535

In [18]:
def SimilarityAllRJ(RatingMatriks, jenis="user-based"):
    if jenis == "user-based":
        jumlah = RatingMatriks.shape[0]
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = SimilarityRJ(RatingMatriks, i, j, jenis=jenis)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=RatingMatriks.index, columns=RatingMatriks.index)
    elif jenis == "item-based":
        jumlah = RatingMatriks.shape[1]
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = SimilarityRJ(RatingMatriks, i, j, jenis=jenis)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=RatingMatriks.columns, columns=RatingMatriks.columns)
    return simMatriksDf

In [18]:
calRJALLUser2 = SimilarityAllRJ(rating_Matriks, jenis="user-based")
calRJALLUser2

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.001848,0.002848,0.003055,0.003339,0.002207,0.002037,0.001435,0.003019,0.003384,0.002178,...,0.001615,0.002864,0.001802,0.002848,0.002053,0.002709,0.002062,0.003173,0.002331,0.001647
2,0.002848,0.006210,0.007282,0.009132,0.003788,0.003321,0.001972,0.007042,0.009494,0.003715,...,0.002330,0.006286,0.002739,0.006209,0.003366,0.005583,0.003389,0.007987,0.004182,0.002397
3,0.003055,0.007282,0.008847,0.011719,0.004167,0.003608,0.002068,0.008523,0.000000,0.004076,...,0.002467,0.007353,0.002932,0.007286,0.003660,0.006431,0.003689,0.009852,0.004646,0.002538
4,0.003339,0.009132,0.011719,0.017522,0.004717,0.000000,0.002196,0.011173,0.000000,0.004587,...,0.002646,0.009259,0.003193,0.009146,0.004077,0.007812,0.004112,0.013636,0.005342,0.002736
5,0.002207,0.003788,0.004167,0.004717,0.002740,0.002481,0.001642,0.004112,0.004819,0.002695,...,0.001883,0.003828,0.002141,0.003795,0.002506,0.003556,0.002519,0.004400,0.002932,0.001927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.002709,0.005583,0.006431,0.007812,0.003556,0.003134,0.001904,0.006250,0.000000,0.003478,...,0.002236,0.005648,0.002611,0.005583,0.003174,0.005076,0.003193,0.006985,0.003886,0.002298
940,0.002062,0.003389,0.003689,0.004112,0.002519,0.002299,0.001560,0.003634,0.004178,0.002481,...,0.001776,0.003411,0.002004,0.003389,0.002320,0.003193,0.002331,0.003859,0.002681,0.001815
941,0.003173,0.007987,0.009852,0.013636,0.004400,0.003771,0.002121,0.009501,0.014388,0.004274,...,0.002542,0.008119,0.003039,0.007991,0.003830,0.006985,0.003859,0.011230,0.004920,0.002623


In [21]:
calRJALLItem2 = SimilarityAllRJ(rating_Matriks, jenis="item-based")
calRJALLItem2

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,0.000652,0.001024,0.001090,0.000902,0.001105,0.001239,0.000724,0.000898,0.000794,0.001095,...,0.001299,0.0,0.00000,0.00000,0.001299,0.000000,0.000000,0.000000,0.001299,0.001299
2,0.001024,0.002375,0.002770,0.001808,0.002865,0.003981,0.001212,0.001795,0.001422,0.002800,...,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.004673,0.004673
3,0.001090,0.002770,0.003322,0.002028,0.003459,0.005229,0.001307,0.002012,0.001555,0.003366,...,0.000000,0.0,0.00000,0.00000,0.006494,0.000000,0.000000,0.000000,0.000000,0.006494
4,0.000902,0.001808,0.002028,0.001460,0.002079,0.002610,0.001045,0.001451,0.001198,0.002045,...,0.000000,0.0,0.00289,0.00289,0.002890,0.000000,0.000000,0.000000,0.002890,0.002890
5,0.001105,0.002865,0.003459,0.002079,0.003610,0.005556,0.001328,0.002062,0.001585,0.003506,...,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00000,0.00000,0.000000,0.166667,0.166667,0.166667,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00000,0.00000,0.000000,0.166667,0.166667,0.166667,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00000,0.00000,0.000000,0.166667,0.166667,0.166667,0.000000,0.000000
1681,0.001299,0.004673,0.000000,0.002890,0.000000,0.000000,0.001618,0.002857,0.002016,0.000000,...,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000


In [24]:
print(calRJALLItem2.shape)

(1682, 1682)


### TOP-K

In [24]:
#use .pkl to find topK

cal_RJ = joblib.load("case/sim/relevantJ/ub/simRJFold5_user-based.joblib")
cal_RJ

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.000995,0.001615,0.001653,0.001828,0.001206,0.001112,0.000792,0.001631,0.001841,0.001198,...,0.001610,0.001876,0.001715,0.001897,0.001776,0.001868,0.001709,0.001934,0.001763,0.001675
2,0.001615,0.004292,0.004564,0.006205,0.002256,0.001949,0.001140,0.004402,0.006361,0.002227,...,0.004252,0.006793,0.005073,0.007084,0.005642,0.006667,0.005021,0.007576,0.005519,0.004734
3,0.001653,0.004564,0.004878,0.006799,0.002326,0.002004,0.001159,0.004691,0.006944,0.002298,...,0.000000,0.000000,0.005455,0.007843,0.006122,0.007380,0.005401,0.008511,0.005976,0.005051
4,0.001828,0.006205,0.006799,0.011230,0.002688,0.002267,0.001242,0.006443,0.011696,0.002651,...,0.006116,0.000000,0.007937,0.014286,0.009434,0.012821,0.007812,0.016667,0.009146,0.007177
5,0.001206,0.002256,0.002326,0.002688,0.001531,0.001383,0.000920,0.002288,0.002722,0.001517,...,0.002247,0.002799,0.002455,0.000000,0.002582,0.002783,0.002444,0.002924,0.002556,0.002375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.001868,0.006667,0.007380,0.012821,0.002783,0.002328,0.001261,0.006977,0.000000,0.000000,...,0.006579,0.015748,0.008811,0.000000,0.010638,0.015370,0.000000,0.021053,0.000000,0.000000
940,0.001709,0.005021,0.005401,0.007812,0.002444,0.002087,0.001186,0.005178,0.008065,0.002409,...,0.004969,0.000000,0.006126,0.009302,0.006981,0.000000,0.006060,0.010256,0.006791,0.005642
941,0.001934,0.007576,0.008511,0.016667,0.002924,0.002432,0.001290,0.007979,0.000000,0.002878,...,0.000000,0.000000,0.010417,0.000000,0.013158,0.021053,0.010256,0.034314,0.012500,0.009146


In [27]:
def TopKTetangga(similarity, k):
    # Mengambil k tetangga terdekat 
    # Mengurutkan similarity dalam urutan desending
    return np.argsort(-similarity)[:k]

In [22]:
def TetanggaK(target_user, target_item, matriks_rating, similarity, k, jenis="user-based"):
    rating_matrix = matriks_rating.to_numpy()  # Convert ke NumPy untuk efisiensi
    if jenis == "user-based":
        # mendapatkan rating dari user yang ditargetkan
        item_ratings = rating_matrix[:, target_item]
        # mencari item yang sudah di rating oleh user
        item_rated_by = np.flatnonzero(item_ratings)
        # print(f"item rated by {len(item_rated_by)}")
        item_rated_by = item_rated_by[item_rated_by != target_user]
        # ambil valaue similarity dari user yang ditargetkan ke item yang sudah di rating oleh user
        sim_scores = similarity.values[target_user, item_rated_by]
        indices = item_rated_by
    else:  # item-based
        user_ratings = rating_matrix[target_user, :]
        # print(f'user rating {len(user_ratings)}')
        items_rated = np.flatnonzero(user_ratings)
        items_rated = items_rated[items_rated != target_item]

        sim_scores = similarity.values[items_rated, target_item]
        indices = items_rated
    #diurutka nilai terbesar ke terkecil
    sorted_idx = np.argsort(-sim_scores)[:k]
    # simScores = sim_scores
    
    # print(f"sorted idx {sorted_idx}")
    # maping tetang ulang berdasarkan index asli 
    top_neighbors = [indices[i] for i in sorted_idx]
    # print(f"top neighbors {top_neighbors}")

    return top_neighbors


## PREDIKSI

In [23]:
def prediksiCF_v2(RatingMatriks, similarityFunction, mean, meanCen, user=2, item=2, k=2, jenis="user-based"):
    # jika jenis user-based
    if jenis == "user-based":
        target_user1 = user
        target_item2 = item
        # ambil mean rating user target
        mean_value = mean.iloc[user]
        # print("mean", mean_value)
    elif jenis == "item-based":
        target_user1 = item
        target_item2 = user
        # ambil mean rating item target
        mean_value = mean.iloc[item]
    # Ambil top-k tetangga
    tetangga = TetanggaK(
        target_user=target_user1,
        target_item=target_item2,
        matriks_rating=RatingMatriks,
        similarity=similarityFunction,
        k=k,
        jenis=jenis
    )
    # print(f"tetangga {len(tetangga)}")
    # menyiapkan pembilang dan penyebut untuk menghitung prediksi rating
    pembilang = 0
    penyebut = 0
    # loop sesuai dengan banyak tetangga
    for tetangga_idx in tetangga:
        if jenis == "user-based":
            # mengambil sim antara user dan tetangga
            sim = similarityFunction.iloc[user, tetangga_idx]
            # print(f"sim {user}-{tetangga_idx} = {sim}")
            mean_cen = meanCen.iloc[tetangga_idx, item]
            # print(f"mean_cen {tetangga_idx}-{item} = {mean_cen}")
        else:  # item-based
            # mengambil sim antara item dan tetangga
            sim = similarityFunction.iloc[tetangga_idx, item]
            # mengambil mean centered rating untuk user target dari tetangga
            mean_cen = meanCen.iloc[user, tetangga_idx]
        pembilang += mean_cen * sim
        # print("pembilang", (pembilang))
        # menghitung penyebut
        penyebut += abs(sim)
        # print("penyebut", penyebut)
    # jika penyebut == 0, maka hitung predikis
    if penyebut != 0:
        prediksi = mean_value + (pembilang / penyebut)
        # print(f"prediks1 {prediksi} = {mean_value} + ({pembilang} / {penyebut})")
    else:
        prediksi = mean_value
    return [item, float(prediksi)]

In [ ]:
def prediksi_semua_matriks(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="user-based"):
    # Salin matriks rating agar tidak mengubah data asli
    hasil_matriks = RatingMatriks.copy()
    num_users, num_items = hasil_matriks.shape
    for user in range(num_users):
        for item in range(num_items):
            if hasil_matriks.iloc[user, item] == 0:
                _, pred_rating = prediksiCF_v2(
                    RatingMatriks=RatingMatriks,
                    similarityFunction=similarityFunction,
                    mean=mean,
                    meanCen=meanCen,
                    user=user,
                    item=item,
                    k=k,
                    jenis=jenis
                )
                hasil_matriks.iloc[user, item] = pred_rating
    return hasil_matriks

In [ ]:
def simpan_semua_prediksi_k(
    RatingMatriks, similarityFunction, mean, meanCen,
    variasi_k, jenis="user-based",
    folder_output="case/prediksi/RJ/ub/5"
):
    for k in variasi_k:
        # Hitung prediksi
        prediksi = prediksi_semua_matriks(
            RatingMatriks=RatingMatriks,
            similarityFunction=similarityFunction,
            mean=mean,
            meanCen=meanCen,
            k=k,
            jenis=jenis
        )

        # Siapkan path file
        path_file = os.path.join(folder_output, f"{k}.joblib")
        os.makedirs(folder_output, exist_ok=True)

        # Simpan hasil prediksi
        joblib.dump(prediksi, path_file)
        print(f"✔️ Prediksi untuk k={k} disimpan di: {path_file}")

In [25]:
simpan_semua_prediksi_k(
    RatingMatriks=rating_Matriks,
    similarityFunction=cal_RJ,
    mean=cal_MeanRatingUser,
    meanCen=cal_MeanCenteredUser,
    variasi_k=[5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200],
    jenis="user-based",
    folder_output="case/prediksi/RJ/ub/5"
)

✔️ Prediksi untuk k=5 disimpan di: case/prediksi/RJ/ub/5\5.joblib
✔️ Prediksi untuk k=10 disimpan di: case/prediksi/RJ/ub/5\10.joblib
✔️ Prediksi untuk k=15 disimpan di: case/prediksi/RJ/ub/5\15.joblib
✔️ Prediksi untuk k=18 disimpan di: case/prediksi/RJ/ub/5\18.joblib
✔️ Prediksi untuk k=20 disimpan di: case/prediksi/RJ/ub/5\20.joblib


KeyboardInterrupt: 

### VARIASI RJ UNTUK USER

In [ ]:
variasiParameterKUser = [5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200]
variasiParameterKItem2= [20]
variasiParameterHybrid = [0.7]

### VARIASI RJ UNTUK ITEM

In [ ]:
variasiParameterKUser = [30]
variasiParameterKItem2= [5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200]
variasiParameterHybrid = [0.7]

### VARIASI RJ UNTUK HYBRID

In [ ]:
variasiParameterKUser = [30]
variasiParameterKItem2 = [30]
variasiParameterHybrid = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

## HYBRID

In [2]:
def HybridFiltering(PrediksiUser, PrediksiItem, gamma=0.7):
    # Mengonversi ke numpy array untuk perhitungan usser dan item
    prediksiUser = np.array(PrediksiUser)
    prediksiItem = np.array(PrediksiItem)
    # Menghitung prediksi hybrid
    rumusHybrid = gamma * prediksiUser + (1 - gamma) * prediksiItem
    # mengembalikan hasil prediksi hybrid
    return rumusHybrid

## TOP-N REKOMENDASI

In [ ]:
def TopNRecommendationPerUser(hybridPredictions, N=5):
    # Membuat dictionary untuk menyimpan rekomendasi teratas per user
    topNPerUser = {}
    # Iterasi setiap user dalam DataFrame untuk mendapatkan N rekomendasi teratas
    for user in hybridPredictions.index:
        # Ambil prediksi untuk user tertentu
        userPredictions = hybridPredictions.loc[user]
        # Menggunakan argsort untuk mengurutkan prediksi rating dari yang tertinggi
        # argsort mengembalikan indeks yang diurutkan dari array
        topNIndices = userPredictions.argsort()[::-1][:N] 
        # Ambil item terkait dengan indeks teratas
        topNItems = userPredictions.index[topNIndices].tolist()
        # Simpan hasil rekomendasi untuk user tersebut
        topNPerUser[user] = topNItems
    # mengembalikan hasil rekomendasi
    return topNPerUser

In [1]:
def FullRecommendation(prediksi_matrix):
    ranking_per_user = np.argsort(-prediksi_matrix, axis=1)+1
    return ranking_per_user


In [ ]:
cal_HybirdVariasi1 = FullRecommendation(variasai1Parameter)
cal_HybirdVariasi1

array([[1450, 1567, 1480, ..., 1629, 1589, 1627],
       [1567, 1621, 1285, ...,  309, 1586, 1627],
       [ 321,  340,  320, ..., 1589, 1627, 1586],
       ...,
       [1450, 1567, 1480, ..., 1586, 1589, 1627],
       [1450, 1567, 1480, ..., 1589, 1586, 1627],
       [1450,  100,   56, ..., 1629, 1589, 1627]], dtype=int64)

In [27]:
simRJUB = joblib.load('case/sim/RJ/ub/simRJFold1_user-based.joblib')
simRJUB

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.001848,0.002848,0.003055,0.003339,0.002207,0.002037,0.001435,0.003019,0.003384,0.002178,...,0.001615,0.002864,0.001802,0.002848,0.002053,0.002709,0.002062,0.003173,0.002331,0.001647
2,0.002848,0.006210,0.007282,0.009132,0.003788,0.003321,0.001972,0.007042,0.009494,0.003715,...,0.002330,0.006286,0.002739,0.006209,0.003366,0.005583,0.003389,0.007987,0.004182,0.002397
3,0.003055,0.007282,0.008847,0.011719,0.004167,0.003608,0.002068,0.008523,0.000000,0.004076,...,0.002467,0.007353,0.002932,0.007286,0.003660,0.006431,0.003689,0.009852,0.004646,0.002538
4,0.003339,0.009132,0.011719,0.017522,0.004717,0.000000,0.002196,0.011173,0.000000,0.004587,...,0.002646,0.009259,0.003193,0.009146,0.004077,0.007812,0.004112,0.013636,0.005342,0.002736
5,0.002207,0.003788,0.004167,0.004717,0.002740,0.002481,0.001642,0.004112,0.004819,0.002695,...,0.001883,0.003828,0.002141,0.003795,0.002506,0.003556,0.002519,0.004400,0.002932,0.001927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.002709,0.005583,0.006431,0.007812,0.003556,0.003134,0.001904,0.006250,0.000000,0.003478,...,0.002236,0.005648,0.002611,0.005583,0.003174,0.005076,0.003193,0.006985,0.003886,0.002298
940,0.002062,0.003389,0.003689,0.004112,0.002519,0.002299,0.001560,0.003634,0.004178,0.002481,...,0.001776,0.003411,0.002004,0.003389,0.002320,0.003193,0.002331,0.003859,0.002681,0.001815
941,0.003173,0.007987,0.009852,0.013636,0.004400,0.003771,0.002121,0.009501,0.014388,0.004274,...,0.002542,0.008119,0.003039,0.007991,0.003830,0.006985,0.003859,0.011230,0.004920,0.002623


In [2]:
import joblib
import os

def process_and_save_mean_ratings(baseFile, saveDirectory):
    # Loop untuk 5 fold
    for fold_number in range(1, 6):
        # Load dataset untuk base dan test
        base_Data, _ = load_Dataset(baseFile, fold_number)
        
        # Convert ke rating matriks
        matriks_rating = ConvertRatingMatriks(base_Data)

       
        matriks = f"{saveDirectory}/u{fold_number}.joblib"
        joblib.dump(matriks_rating, matriks)
        print(f"✔️ matriks di: {matriks}")
        # joblib



# Contoh penggunaan
baseFile = "ml-100k"  # Ganti dengan path file data Anda
matriksRatintg = "case/ratingMatriks"

process_and_save_mean_ratings(baseFile, matriksRatintg)

# # # Menyimpan mean rating untuk item-based







NameError: name 'load_Dataset' is not defined

In [16]:
dataMatriks = joblib.load("case/ratingMatriks/u1.joblib")
dataMatriks

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
dataopen = joblib.load("case/prediksi/Jac/ub/1/5.joblib")
dataopen

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.218691,4.000000,1.000000,5.000000,3.728815,...,2.534423,3.544426,3.222151,2.222151,3.469500,1.559986,3.559986,2.559986,3.700818,3.315551
2,4.000000,3.222722,2.963838,3.247616,3.891988,4.830095,3.889122,4.532790,4.440090,2.000000,...,2.652941,3.662944,3.340670,2.340670,3.588018,1.678505,3.678505,2.678505,3.819337,3.434069
3,3.814737,1.785034,2.182020,3.027970,3.085057,2.371788,3.193913,3.120864,3.799709,3.388455,...,1.852941,2.862944,2.540670,1.540670,2.788018,0.878505,2.878505,1.878505,3.019337,2.634069
4,5.046868,3.067127,4.001402,3.844595,3.888865,4.428490,4.923513,4.586607,4.893147,4.506614,...,3.210084,4.220087,3.897813,2.897813,4.145161,2.235648,4.235648,3.235648,4.376480,3.991212
5,3.750135,2.815731,2.322002,2.763512,3.070289,2.139646,3.667401,3.813842,2.751794,3.071424,...,1.808985,2.818988,2.496714,1.496714,2.744062,2.956044,2.956044,2.956044,2.975381,2.590113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.776029,3.958011,4.049760,3.642780,4.181584,5.034501,3.625027,4.829840,5.000000,4.435371,...,3.118247,4.128250,3.805976,2.805976,4.053325,2.143811,4.143811,3.143811,4.284643,3.899376
940,3.403160,3.252267,2.487956,2.000000,2.443996,2.845468,4.000000,5.000000,3.000000,3.779176,...,2.310885,3.320888,2.998614,1.998614,3.245962,1.336449,3.336449,2.336449,3.477281,3.092013
941,5.000000,3.576549,3.975729,2.953050,3.741924,4.123861,4.000000,4.972952,3.971818,3.854685,...,2.898396,3.908399,3.586124,2.586124,3.833473,1.923959,3.923959,2.923959,4.064792,3.679524


In [4]:
import joblib
dataoPEN3d = joblib.load("case/prediksiModel/Jac/ub/1/5.joblib")
dataoPEN3d

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.218691,4.000000,1.000000,5.000000,3.728815,...,2.534423,3.544426,3.222151,2.222151,3.469500,1.559986,3.559986,2.559986,3.700818,3.315551
2,4.000000,3.222722,2.963838,3.247616,3.891988,4.830095,3.889122,4.532790,4.440090,2.000000,...,2.652941,3.662944,3.340670,2.340670,3.588018,1.678505,3.678505,2.678505,3.819337,3.434069
3,3.814737,1.785034,2.182020,3.027970,3.085057,2.371788,3.193913,3.120864,3.799709,3.388455,...,1.852941,2.862944,2.540670,1.540670,2.788018,0.878505,2.878505,1.878505,3.019337,2.634069
4,5.046868,3.067127,4.001402,3.844595,3.888865,4.428490,4.923513,4.586607,4.893147,4.506614,...,3.210084,4.220087,3.897813,2.897813,4.145161,2.235648,4.235648,3.235648,4.376480,3.991212
5,3.750135,2.815731,2.322002,2.763512,3.070289,2.139646,3.667401,3.813842,2.751794,3.071424,...,1.808985,2.818988,2.496714,1.496714,2.744062,2.956044,2.956044,2.956044,2.975381,2.590113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.776029,3.958011,4.049760,3.642780,4.181584,5.034501,3.625027,4.829840,5.000000,4.435371,...,3.118247,4.128250,3.805976,2.805976,4.053325,2.143811,4.143811,3.143811,4.284643,3.899376
940,3.403160,3.252267,2.487956,2.000000,2.443996,2.845468,4.000000,5.000000,3.000000,3.779176,...,2.310885,3.320888,2.998614,1.998614,3.245962,1.336449,3.336449,2.336449,3.477281,3.092013
941,5.000000,3.576549,3.975729,2.953050,3.741924,4.123861,4.000000,4.972952,3.971818,3.854685,...,2.898396,3.908399,3.586124,2.586124,3.833473,1.923959,3.923959,2.923959,4.064792,3.679524
942,4.878181,3.814061,3.478054,4.069125,4.053574,4.071380,4.239001,5.162838,4.500917,4.627949,...,3.118764,4.128767,3.806493,2.806493,4.053841,2.144327,4.144327,3.144327,4.285160,3.899892


In [12]:
dataoPEN2 = joblib.load("case/prediksiModel/Jac/ub/1/5.joblib")
dataoPEN2

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,3.681481,3.681481,3.681481,3.681481,3.681481,5.000000,3.681481,3.681481,3.681481,3.000000,...,3.681481,3.681481,3.681481,3.681481,3.681481,3.681481,3.681481,3.681481,3.681481,3.681481
2,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,...,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000
3,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
4,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,...,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143,4.357143
5,4.000000,3.000000,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044,...,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044,2.956044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,...,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306,4.265306
940,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,...,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944,3.457944
941,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,...,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455,4.045455
942,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,...,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823,4.265823


,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.218691,4.000000,1.000000,5.000000,3.728815,...,2.534423,3.544426,3.222151,2.222151,3.469500,1.559986,3.559986,2.559986,3.700818,3.315551
2,4.000000,3.222722,2.963838,3.247616,3.891988,4.830095,3.889122,4.532790,4.440090,2.000000,...,2.652941,3.662944,3.340670,2.340670,3.588018,1.678505,3.678505,2.678505,3.819337,3.434069
3,3.814737,1.785034,2.182020,3.027970,3.085057,2.371788,3.193913,3.120864,3.799709,3.388455,...,1.852941,2.862944,2.540670,1.540670,2.788018,0.878505,2.878505,1.878505,3.019337,2.634069
4,5.046868,3.067127,4.001402,3.844595,3.888865,4.428490,4.923513,4.586607,4.893147,4.506614,...,3.210084,4.220087,3.897813,2.897813,4.145161,2.235648,4.235648,3.235648,4.376480,3.991212
5,3.750135,2.815731,2.322002,2.763512,3.070289,2.139646,3.667401,3.813842,2.751794,3.071424,...,1.808985,2.818988,2.496714,1.496714,2.744062,2.956044,2.956044,2.956044,2.975381,2.590113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.776029,3.958011,4.049760,3.642780,4.181584,5.034501,3.625027,4.829840,5.000000,4.435371,...,3.118247,4.128250,3.805976,2.805976,4.053325,2.143811,4.143811,3.143811,4.284643,3.899376
940,3.403160,3.252267,2.487956,2.000000,2.443996,2.845468,4.000000,5.000000,3.000000,3.779176,...,2.310885,3.320888,2.998614,1.998614,3.245962,1.336449,3.336449,2.336449,3.477281,3.092013
941,5.000000,3.576549,3.975729,2.953050,3.741924,4.123861,4.000000,4.972952,3.971818,3.854685,...,2.898396,3.908399,3.586124,2.586124,3.833473,1.923959,3.923959,2.923959,4.064792,3.679524
942,4.878181,3.814061,3.478054,4.069125,4.053574,4.071380,4.239001,5.162838,4.500917,4.627949,...,3.118764,4.128767,3.806493,2.806493,4.053841,2.144327,4.144327,3.144327,4.285160,3.899892
